In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

## Binary Classifier (call/no call) using the Training data

In [2]:
from birdclef.utils import get_spark

spark = get_spark(cores=4, memory="10g")
# Load train_postprocessed/v7 dataset
df = spark.read.parquet("../data/processed/birdclef-2023/train_postprocessed/v7")
df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/06 14:50:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- track_stem: string (nullable = true)
 |-- track_type: string (nullable = true)
 |-- start_time: long (nullable = true)
 |-- species: string (nullable = true)
 |-- embedding: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- prediction_vec: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [3]:
# Import background noise dataset
back_noise_df = spark.read.parquet("../data/processed/birdclef-2023/background_noise")
back_noise_df.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- emb: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- call: double (nullable = true)
 |-- no_call: double (nullable = true)



In [4]:
back_noise_df.count()

8136

In [5]:
from pyspark.sql.functions import lit

# Import background noise dataset
back_noise_df = spark.read.parquet("../data/processed/birdclef-2023/background_noise")
back_noise_df.printSchema()

# Add a 'species' column to back_noise_df with 'no_call' as values
back_noise_df = back_noise_df.withColumn("species", lit("no_call"))
back_noise_df = back_noise_df.withColumnRenamed("emb", "embedding")

# Make sure 'species' is the first column to math the structure of negatives DF
back_noise_sub = back_noise_df.select("species", "embedding")
# negatives_sub = negatives.select("species", "embedding")

# Union the two DFs
# negatives_df = negatives_sub.union(back_noise_sub)

# Select positive samples and Union with negatives
positives_sub = df.select("species", "embedding")
positives_df = positives_sub.withColumn("species", lit("call"))
binary_df = back_noise_sub.union(positives_df)
binary_df.show()

root
 |-- row_id: string (nullable = true)
 |-- emb: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- call: double (nullable = true)
 |-- no_call: double (nullable = true)

+-------+--------------------+
|species|           embedding|
+-------+--------------------+
|no_call|[0.92088675498962...|
|no_call|[0.84365564584732...|
|no_call|[1.26694309711456...|
|no_call|[1.38729619979858...|
|no_call|[0.67062968015670...|
|no_call|[0.55484676361083...|
|no_call|[0.76140671968460...|
|no_call|[1.05057728290557...|
|no_call|[1.10778868198394...|
|no_call|[0.97304064035415...|
|no_call|[0.80366438627243...|
|no_call|[0.83019882440567...|
|no_call|[1.27283263206481...|
|no_call|[1.18015515804290...|
|no_call|[1.46877062320709...|
|no_call|[1.15537261962890...|
|no_call|[1.25248849391937...|
|no_call|[1.28011035919189...|
|no_call|[1.57398808002471...|
|no_call|[1.14994704723358...|
+-------+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql import Window, functions as F

# Check the number of samples for each label
counts = binary_df.groupBy("species").agg(F.count("*").alias("n")).orderBy(F.desc("n"))
counts.show(n=5)

+-------+------+
|species|     n|
+-------+------+
|   call|255372|
|no_call|  8136|
+-------+------+

